In [1]:
import os
import numpy as np
import pandas as pd

from mpire import WorkerPool

RAW_SNP_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/snpDir/"
RAW_RES_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/Result/"
NEW_SNP_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/offspring/01/snpDir/"
NEW_RES_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/offspring/01/Result/"
REF_PATH = "/data1/zhoujb/cowpeaSequence/refernce/"
TEMP_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/templateSNP/"

In [2]:
genotype_hap_id = pd.read_table(os.path.join(NEW_RES_PATH, "Genotype_by_Hap_01.tsv"), index_col="Region_index")

haplotype_score_info = pd.read_table(os.path.join(RAW_RES_PATH, "Haplotype_Score_Info.tsv"), index_col="Haplotype_ID")
haplotype_score_info = haplotype_score_info[['Hap_Score.by_Total_pop','Hap_Score.by_VL_pop', 'Hap_Score.by_G_pop', 'Hap_Score.by_VC_pop']].copy()

name_list = []
with open("/data2/zhoujb/project/cowpea_project/runWES/P01_10_res/config_bam") as name_f:
    for line in name_f:
        if line.startswith("01"):
            name_list.append(line.split("\t")[0])


output_file = open(os.path.join(NEW_RES_PATH, "Samples_Score_Info_01.tsv"), "w")
print("\t".join(["Sample", "Mis_Hap_Num", 
                 "Hap_Score.by_Total_pop", "Hap_Score.by_VL_pop", "Hap_Score.by_G_pop", "Hap_Score.by_VC_pop"]), file=output_file)

for index in name_list:
    tmp_hap_id = [x for x in genotype_hap_id[index].values if x != "-"]
    mis_hap_num = len([x for x in genotype_hap_id[index].values if x == "-"])
    tmp_score = haplotype_score_info.loc[tmp_hap_id]
    tmp_sum = tmp_score.sum()
    final_score = [tmp_sum['Hap_Score.by_Total_pop']/854.768527747741*100, 
                   tmp_sum['Hap_Score.by_VL_pop']/952.5366599029439*100, 
                   tmp_sum['Hap_Score.by_G_pop']/547.8057376793577*100, 
                  tmp_sum['Hap_Score.by_VC_pop']/989.25449595043*100]
    
    print("\t".join([index, str(mis_hap_num)]+[str(x) for x in final_score]), file=output_file)

output_file.close()